<a href="https://colab.research.google.com/github/bruno-raffa/Quantum-exercises/blob/main/LineUp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Line-up Optimization
The aim of this demo, developed by Aitzol Iturrospe (www.spdtek.com), is to optimize the initial line-up of Liverpool FC. The goal is to select players maximizing the sum of their ratings. The eleven football players are divided into several positions in accordance with the team formation. In addition to one goalkeeper (GK), the players are divided into three main positions, defenders (D), midfielders (M), and forward/strikers (FW). Each major position can be subdivided into several more specific positions, such as



*   central defender (DC)
*   left wing defender (DL)
*   right wing defender (DR)
*   defensive midfielder (DM)
*   central midfielder (CM)
*   attack midfielder (AM)
*   right wing forward (FWR)
*   left wing forward (FWL) or
forward/striker (FW)


The problem is stated as a constrained quadratic model **(CQM)** and it is solved in a D-Wave Leap’s Hybrid Solver.

In [ ]:
! pip install dwave-ocean-sdk
! dwave setup

In [ ]:
import matplotlib
import numpy as np
from dimod import ConstrainedQuadraticModel, BinaryQuadraticModel, QuadraticModel
from dwave.system import LeapHybridCQMSampler
import pandas as pd
import urllib.request

try:
    import matplotlib.pyplot as plt
except ImportError:
    matplotlib.use("agg")
    import matplotlib.pyplot as plt



In [ ]:
# Read players rating from CSV
filename = 'https://raw.githubusercontent.com/dwave-examples/Line-up-optimization/master/players.csv'
column_names = ["Variable", "Player", "Position", "Rating"]
players_df = pd.read_csv(filename, names=column_names)
ratings = players_df['Rating'].tolist()

In [ ]:
robertson = [7,11,16]
jota = [27,32,34,37,41]

ratings_robertson = [e for i, e in enumerate(ratings) if i in robertson]
ratings_jota = [e for i, e in enumerate(ratings) if i in jota]

In [ ]:
n_var = 43
print("\nBuilding a CQM for {} items.".format(str(n_var)))

cqm = ConstrainedQuadraticModel()
obj = BinaryQuadraticModel(vartype='BINARY')

#Constraints to setup a proper team
constraint1 = QuadraticModel()
constraint2 = QuadraticModel()
constraint3 = QuadraticModel()
constraint4 = QuadraticModel()
constraint6 = QuadraticModel()
constraint7 = QuadraticModel()
constraint8 = QuadraticModel()
constraint9 = QuadraticModel()

#Constraints to avoid player in multiple roles
constraint10 = QuadraticModel()
constraint11 = QuadraticModel()


for i in range(n_var):
    # Objective is to maximize the total costs
    obj.add_variable(i)
    obj.set_linear(i, -ratings[i])

    # Constraint 1
    constraint1.add_variable('BINARY', i)
    constraint1.set_linear(i, 1)


# Constraint 2
for i in range(0,2):
  constraint2.add_variable('BINARY', i)
  constraint2.set_linear(i, 1)

# Constraint 3
for i in range(2,7):
  constraint3.add_variable('BINARY', i)
  constraint3.set_linear(i, 1)

# Constraint 4
for i in range(7,8):
  constraint4.add_variable('BINARY', i)
  constraint4.set_linear(i, 1)

# Constraint 5
for i in range(8,11):
  constraint5.add_variable('BINARY', i)
  constraint5.set_linear(i, 1)

# Constraint 6
for i in range(38,42):
  constraint6.add_variable('BINARY', i)
  constraint6.set_linear(i, 1)

# Constraint 7
for i in range(16,28):
  constraint7.add_variable('BINARY', i)
  constraint7.set_linear(i, 1)


# Constraint 8
for i in range(33,36):
  constraint8.add_variable('BINARY', i)
  constraint8.set_linear(i, 1)

# Constraint 9
for i in range(36,38):
  constraint9.add_variable('BINARY', i)
  constraint9.set_linear(i, 1)

# Constraint single position for each player
for i in range(n_var):
  if i in robertson:
    constraint10.add_variable('BINARY', i)
    constraint10.set_linear(i, 1)
  if i in jota:
    constraint11.add_variable('BINARY', i)
    constraint11.set_linear(i, 1)

cqm.set_objective(obj)

cqm.add_constraint(constraint1 == 11, label='11_players') 
cqm.add_constraint(constraint2 == 1, label='1 Goalkeeper') 
cqm.add_constraint(constraint3 == 2, label='2 Defender Central') 
cqm.add_constraint(constraint4 == 1, label='1 Defender Left') 
cqm.add_constraint(constraint5 == 1, label='1 Defender Righ') 
cqm.add_constraint(constraint6 == 1, label='1 Forward Striker') 
cqm.add_constraint(constraint7 == 3, label='3 Midfielder') 
cqm.add_constraint(constraint8 == 1, label='1 Forward Left') 
cqm.add_constraint(constraint9 == 1, label='1 Forward Right') 
cqm.add_constraint(constraint10 == 1, label='Robertson')
cqm.add_constraint(constraint11 == 1, label='Jota') 


Building a CQM for 43 items.


'Jota'

In [ ]:
sampler = LeapHybridCQMSampler()
print("Submitting CQM to solver {}.".format(sampler.solver.name))
sampleset = sampler.sample_cqm(cqm, label='Example - Lineup')

Submitting CQM to solver hybrid_constrained_quadratic_model_version1.


In [ ]:
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)
if not len(feasible_sampleset):
    raise ValueError("No feasible solution found")
best = feasible_sampleset.first
selected_item_indices = [key for key, val in best.sample.items() if val==1.0]
selected_item_indices

[0, 5, 6, 7, 10, 17, 20, 27, 33, 36, 38]

In [ ]:
# Print results for best line-up
lineup_df = pd.DataFrame(best.sample.items())
lineup_df.columns = ['Variable', 'Selected']
lineup_df = lineup_df[(lineup_df['Selected'] == 1)]
result = players_df.filter(items = lineup_df.Variable.tolist(), axis=0).drop('Variable', axis=1)
(result)

,Player,Position,Rating
0,Alisson,GK,6.81
5,Phillips,DC,7.24
6,Fabinho,DC,7.11
7,Robertson,DL,6.85
10,Milner,DR,8.15
17,Williams,CM,7.77
20,Thiago,CM,7.38
27,Jota,CM,9.39
33,Mane,FWL,7.56
36,Salah,FWR,7.42


In [ ]:
print(result['Rating'].sum())

82.67
